In [216]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle

In [217]:
rdir_path = './newData'

In [218]:
imgs = []
labels = []
label = -1
onlyDirs = [f for f in os.listdir(rdir_path) if os.path.isdir(os.path.join(rdir_path, f))]
for directory in onlyDirs:
    label = label + 1
    path = rdir_path+'/'+directory
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    for file in onlyfiles:
        imgPath = path+'/'+file
        imgs.append(imgPath)
        labels.append(label)

In [219]:
print(len(imgs))

179


In [220]:
def getdescriptors(imgs):
    descriptors = np.asarray([])
    extractor = cv2.SIFT_create()
    #i = 0
    for img in imgs:
        gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        kp, desc = extractor.detectAndCompute(gray, None)
        if type(desc) == np.ndarray :
            if descriptors.shape[0] == 0:
                descriptors = desc
            else:
                descriptors = np.concatenate((descriptors, desc), axis=0)
    return descriptors

In [221]:
def getfeatures(imgs,vocab):
    bbok = []
    extractor = cv2.SIFT_create()
    for img in imgs:
        bok = [0]*nClusters
        keypoints, descriptors = extractor.detectAndCompute(img, None)
        if type(descriptors) == np.ndarray:
            pred = vocab.predict(descriptors)
            for ele in pred:
                bok[ele] = bok[ele] + 1
        bbok.append(bok)
    return  np.asarray(bbok)

In [222]:
trainImgPaths,testImgPaths, trainLabels, testLabels = train_test_split(imgs, labels, 
                                                train_size=0.8, random_state=42,shuffle = True,stratify = labels)

In [243]:
nClusters = 80
train_imgs = []
for imgName in trainImgPaths:
    img = cv2.imread(imgName)
    train_imgs.append(img)
train_imgs = np.asarray(train_imgs)

In [253]:
descriptors = getdescriptors(train_imgs)
kmeans = KMeans(n_clusters = nClusters)
kmeans.fit(descriptors)

KMeans(n_clusters=80)

In [254]:
train_features = getfeatures(trainImgs,kmeans)

In [255]:
model = svm.SVC(C=0.1,kernel='linear')
model.fit(train_features,trainLabels)

SVC(C=0.1, kernel='linear')

In [247]:
testImgs = []
for imgName in testImgPaths:
    img = cv2.imread(imgName)
    testImgs.append(img)
testImgs = np.asarray(testImgs)
test_features = getfeatures(testImgs,kmeans)

In [248]:
acc = model.score(test_features,testLabels)
print(acc)

0.9444444444444444


In [249]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10)
neigh.fit(train_features,trainLabels)

KNeighborsClassifier(n_neighbors=10)

In [250]:
acck = neigh.score(test_features,testLabels)
print(acck)

0.9166666666666666


In [251]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0, max_iter = 5000).fit(train_features,trainLabels)

In [252]:
acclr = clf.score(test_features,testLabels)
print(acclr)

0.9444444444444444
